Импортируем библиотеки:

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Загрузим данные:

In [2]:
data_train = pd.read_csv(r"D:/titanic/train.csv")
data_test = pd.read_csv(r"D:/titanic/test.csv")

Узнаем, что обозначает каждый столбец в датасете:

1. PassengerID - Id пассажира
2. Survived - выжил пассажир или нет (0 - нет, 1 - да)
3. Pclass - класс билета (1 - первый, 2 - второй, 3 - третий)
4. Name - имя
5. Sex - пол (male - мужчина, female - женщина)
6. Age - возраст
7. SibSp - количество братьев и сестёр / супругов на борту
8. Parch - количество детей / родителей на борту
9. Ticket - номер билета
10. Fare - пассажирский тариф
11. Cabin - номер кабины
12. Embarked - порт посадки (C = Cherbourg, Q = Queenstown, S = Southampton)

Посмотрим на данные:

In [3]:
print(data_train.shape[0], "- количество примеров")
print(data_train.shape[1], "- количество признаков")

891 - количество примеров
12 - количество признаков


In [4]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
print(data_test.shape[0], "- количество примеров")
print(data_test.shape[1], "- количество признаков")

418 - количество примеров
11 - количество признаков


In [6]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Посмотрим, есть ли пропущенные данные:

In [7]:
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
data_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Очень большое количество данных о кабине пассажира пропущено. Есть много пропусков о возрасте пассажира. Также наблюдаем единичные пропуски по признакам, овтечающим за тариф пассажира и порт его отправки. Далее мы заполним пропуски, но сначала посмотрим, как признаки влияют на вероятность выживания пассажира.

#### 1. Pclass

In [9]:
data_train[["Pclass", "Survived"]].groupby(["Pclass"], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


Вероятность выжить была больше у пассажиров первого класса. Только четверть пассажиров третьего класса выжила.

#### 2. Sex

In [10]:
data_train[["Sex", "Survived"]].groupby(["Sex"], as_index=False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


Из всех женщин на борту выжило порядка 75%. Вероятность выжить у мужчин в разы меньше.

#### 3. SibSp

In [11]:
data_train[["SibSp", "Survived"]].groupby(["SibSp"], as_index=False).mean()

,SibSp,Survived
0,0,0.345395
1,1,0.535885
2,2,0.464286
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


При путешествии с одним или двумя братьями и / или сёстрами / супругами наблюдается повышенная вероятность выжить. При это наблюдается, как с увеличением этого параметра резко уменьшаются шансы на выживание.

#### 4. Parch

In [12]:
data_train[["Parch", "Survived"]].groupby(["Parch"], as_index=False).mean()

,Parch,Survived
0,0,0.343658
1,1,0.550847
2,2,0.500000
3,3,0.600000
4,4,0.000000
5,5,0.200000
6,6,0.000000


Люди, путешествующие с количеством детей или родителей от одного до трех, имели хорошие шансы на выживание.

#### 5. Embarked

In [13]:
data_train[["Embarked", "Survived"]].groupby(["Embarked"], as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


Из пассажиров, которые начали своё путешествие из Cherbourg, выжило порядка 55%. Примерно в 1.5 раза меньше была вероятность выжишь у пассажиров, которые зашли на борт в других городах.

Объединим данные для их дальнейшей подготовки:

In [14]:
data = pd.concat([data_train, data_test], ignore_index=True, sort=False)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Имена не несут в себе никакой информации, оставим от них только дополнительное слово, которое используется для обращения:

In [15]:
data["Name"] = data["Name"].apply(lambda name: name.split(',')[1].split('.')[0].strip())
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
data["Name"].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Mlle              2
Ms                2
the Countess      1
Don               1
Dona              1
Mme               1
Lady              1
Sir               1
Capt              1
Jonkheer          1
Name: Name, dtype: int64

Выделим пять групп: мужчины; женщины (которые точно замужем); женщины (о замужестве которых нам неизвестно или мы точно знаем, что они не замужем); мальчики; другие.

In [17]:
data["Name"] = data["Name"].replace(["Rev", "Dr", "Col", "Major", "Mlle", "Jonkheer", 
                                     "Lady", "Dona", "Don", "Mme", "the Countess", "Capt", "Sir"], "Other")

data["Name"] = data["Name"].replace("Ms", "Miss")
data["Name"].value_counts()

Mr        757
Miss      262
Mrs       197
Master     61
Other      32
Name: Name, dtype: int64

In [18]:
data["Name"] = data["Name"].map({"Mr": 0, "Mrs": 1, "Miss": 2, "Master": 3, "Other": 4}).astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,2,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,0,male,35.0,0,0,373450,8.0500,NaN,S


Переведём категориальные признаки пола в числовые:

In [19]:
data["Sex"] = data["Sex"].map({"male": 0, "female": 1}).astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,1,1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,2,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,1,1,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,0,0,35.0,0,0,373450,8.0500,NaN,S


Ещё раз посмотрм на признаки, у которых есть пропущенные значения:

In [20]:
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

Заполним признак Embarked самым часто встречающимся значением. Также переведем категориальные признаки в числовые:

In [21]:
data["Embarked"].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [22]:
data["Embarked"] = data["Embarked"].fillna("S")

In [23]:
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2}).astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1.0,1,1,1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1.0,3,2,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1.0,1,1,1,35.0,1,0,113803,53.1000,C123,0
4,5,0.0,3,0,0,35.0,0,0,373450,8.0500,NaN,0


Теперь заполним пропущенные значения возраста у пассажиров. Для этого посмотрим на медиану возраста при различных значениях номеров класса и пола:

In [24]:
group = data.groupby(["Sex", "Pclass"]).median()
group["Age"]

Sex  Pclass
0    1         42.0
     2         29.5
     3         25.0
1    1         36.0
     2         28.0
     3         22.0
Name: Age, dtype: float64

In [25]:
data["Age"] = data.groupby(["Sex", "Pclass"])["Age"].apply(lambda x: x.fillna(x.median()))
data["Age"] = data["Age"].astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0


Теперь заполним пропущенное значения в колонке Fare. На пассажирский тариф могут повлиять количество братьев и сестер / супругов, количество родителей / детей и номер класса. Узнаем, в каком классе пропущено значение, и заполнил его медианой в соответствии с вышеупомянутыми признаками:

In [26]:
data["Fare"] = data["Fare"].fillna(-1.0)
data[data["Fare"] == -1.0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,NaN,3,0,0,60,0,0,3701,-1.0,NaN,0


In [27]:
group = data.groupby(["SibSp", "Parch", "Pclass"])
fare_value = group["Fare"].median()[0][0][3]

data["Fare"] = data["Fare"].replace(-1.0, fare_value)

In [28]:
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
dtype: int64

Теперь разобьём пассажиров на разные группы по возрасту и стоимости тарифа:

In [29]:
data["Age"].describe([0.1, 0.25, 0.5, 0.75, 0.9])

count    1309.000000
mean       29.237586
std        13.229135
min         0.000000
10%        16.000000
25%        22.000000
50%        26.000000
75%        36.000000
90%        48.000000
max        80.000000
Name: Age, dtype: float64

In [30]:
data["AgeGroup"] = pd.qcut(data["Age"], q=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]"
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]"
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]"
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]"
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]"


In [31]:
data[["AgeGroup", "Survived"]].groupby(["AgeGroup"], as_index=False).mean()

,AgeGroup,Survived
0,"(-0.001, 16.0]",0.550000
1,"(16.0, 22.0]",0.393064
2,"(22.0, 26.0]",0.225275
3,"(26.0, 36.0]",0.457014
4,"(36.0, 48.0]",0.340741
5,"(48.0, 80.0]",0.387500


Самая большая вероятность выжить была у детей до 16 лет.

In [32]:
data.loc[data["Age"] <= 16, "Age_Group"] = 0
data.loc[(data["Age"] > 16) & (data["Age"] <= 22), "Age_Group"] = 1
data.loc[(data["Age"] > 22) & (data["Age"] <= 26), "Age_Group"] = 2
data.loc[(data["Age"] > 26) & (data["Age"] <= 36), "Age_Group"] = 3
data.loc[(data["Age"] > 36) & (data["Age"] <= 48), "Age_Group"] = 4
data.loc[data["Age"] > 48, "Age_Group"] = 5

data["Age_Group"] = data["Age_Group"].astype(int)

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Age_Group
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]",1
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]",4
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]",2
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]",3
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]",3


In [33]:
data["Fare"].describe([0.1, 0.25, 0.5, 0.75, 0.9])

count    1309.000000
mean       33.276044
std        51.743657
min         0.000000
10%         7.570000
25%         7.895800
50%        14.454200
75%        31.275000
90%        78.019980
max       512.329200
Name: Fare, dtype: float64

In [34]:
data["FareGroup"] = pd.qcut(data["Fare"], q=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Age_Group,FareGroup
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]",1,"(-0.001, 7.57]"
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]",4,"(31.275, 78.02]"
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]",2,"(7.896, 14.454]"
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]",3,"(31.275, 78.02]"
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]",3,"(7.896, 14.454]"


In [35]:
data[["FareGroup", "Survived"]].groupby(["FareGroup"], as_index=False).mean()

,FareGroup,Survived
0,"(-0.001, 7.57]",0.141304
1,"(7.57, 7.896]",0.236641
2,"(7.896, 14.454]",0.303571
3,"(14.454, 31.275]",0.441048
4,"(31.275, 78.02]",0.492188
5,"(78.02, 512.329]",0.758621


Здесь отлично прослеживается зависимость шанса выжить от пассажирского тарифа. С увеличением стоимости тарифа увеличивается и вероятность выжить.

In [36]:
data.loc[data["Fare"] <= 7.57, "Fare_Group"] = 0
data.loc[(data["Fare"] > 7.57) & (data["Fare"] <= 7.896), "Fare_Group"] = 1
data.loc[(data["Fare"] > 7.896) & (data["Fare"] <= 14.454), "Fare_Group"] = 2
data.loc[(data["Fare"] > 14.454) & (data["Fare"] <= 31.275), "Fare_Group"] = 3
data.loc[(data["Fare"] > 31.275) & (data["Fare"] <= 78.02), "Fare_Group"] = 4
data.loc[data["Fare"] > 78.02, "Fare_Group"] = 5

data["Fare_Group"] = data["Fare_Group"].astype(int)

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Age_Group,FareGroup,Fare_Group
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]",1,"(-0.001, 7.57]",0
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]",4,"(31.275, 78.02]",4
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]",2,"(7.896, 14.454]",2
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]",3,"(31.275, 78.02]",4
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]",3,"(7.896, 14.454]",2


На основе признаков SibSp и Parch создадим признак, которой будет отражать, путешествует человек в одиночку или нет:

In [37]:
data["Relatives"] = data["SibSp"] + data["Parch"]
data["Relatives"] = data["Relatives"].astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Age_Group,FareGroup,Fare_Group,Relatives
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]",1,"(-0.001, 7.57]",0,1
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]",4,"(31.275, 78.02]",4,1
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]",2,"(7.896, 14.454]",2,0
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]",3,"(31.275, 78.02]",4,1
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]",3,"(7.896, 14.454]",2,0


In [38]:
data[["Relatives", "Survived"]].groupby(["Relatives"], as_index=False).mean()

,Relatives,Survived
0,0,0.303538
1,1,0.552795
2,2,0.578431
3,3,0.724138
4,4,0.200000
5,5,0.136364
6,6,0.333333
7,7,0.000000
8,10,0.000000


In [39]:
data.loc[data["Relatives"] == 0, "IsAlone"] = 1
data.loc[data["Relatives"] > 0, "IsAlone"] = 0

data["IsAlone"] = data["IsAlone"].astype(int)

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Age_Group,FareGroup,Fare_Group,Relatives,IsAlone
0,1,0.0,3,0,0,22,1,0,A/5 21171,7.2500,NaN,0,"(16.0, 22.0]",1,"(-0.001, 7.57]",0,1,0
1,2,1.0,1,1,1,38,1,0,PC 17599,71.2833,C85,1,"(36.0, 48.0]",4,"(31.275, 78.02]",4,1,0
2,3,1.0,3,2,1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,"(22.0, 26.0]",2,"(7.896, 14.454]",2,0,1
3,4,1.0,1,1,1,35,1,0,113803,53.1000,C123,0,"(26.0, 36.0]",3,"(31.275, 78.02]",4,1,0
4,5,0.0,3,0,0,35,0,0,373450,8.0500,NaN,0,"(26.0, 36.0]",3,"(7.896, 14.454]",2,0,1


In [40]:
data[["IsAlone", "Survived"]].groupby(["IsAlone"], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


Теперь удалим ненужные признаки:

In [41]:
data.drop(columns=["Age", "SibSp", "Parch", "Ticket", "Cabin", "AgeGroup", "FareGroup", "Relatives", "Fare"], inplace=True)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Embarked,Age_Group,Fare_Group,IsAlone
0,1,0.0,3,0,0,0,1,0,0
1,2,1.0,1,1,1,1,4,4,0
2,3,1.0,3,2,1,0,2,2,1
3,4,1.0,1,1,1,0,3,4,0
4,5,0.0,3,0,0,0,3,2,1


Разобьём данные на тренировочные и тестовые:

In [42]:
train_data = data[data["Survived"].notna()]

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Embarked,Age_Group,Fare_Group,IsAlone
0,1,0.0,3,0,0,0,1,0,0
1,2,1.0,1,1,1,1,4,4,0
2,3,1.0,3,2,1,0,2,2,1
3,4,1.0,1,1,1,0,3,4,0
4,5,0.0,3,0,0,0,3,2,1


In [43]:
test_data = data.drop(data[data["Survived"] >= 0].index)
test_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Embarked,Age_Group,Fare_Group,IsAlone
891,892,NaN,3,0,0,2,3,1,1
892,893,NaN,3,1,1,0,4,0,0
893,894,NaN,2,0,0,2,5,2,1
894,895,NaN,3,0,0,0,3,2,1
895,896,NaN,3,1,1,0,1,2,0


В качестве классификатора будем использовать случайный лес. Подберём параметры:

In [44]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier

In [45]:
y = train_data["Survived"]

features = ["Pclass", "Name", "Sex", "Embarked", "Age_Group", "Fare_Group", "IsAlone"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [46]:
kf = KFold(n_splits=5)

n_estimators_array = [5, 10, 20, 50, 100, 200]
criterion_array = ["gini", "entropy"]
max_depth_array = [None, 10, 25, 50]

params_grid = dict(n_estimators=n_estimators_array, criterion=criterion_array, max_depth=max_depth_array, random_state=[42])

grid_search = GridSearchCV(RandomForestClassifier(), params_grid, cv=kf, scoring="roc_auc")

grid_search_result = grid_search.fit(X, y)

In [47]:
print ("Лучший результат: ", grid_search_result.best_score_)
print ("Лучшая количество деревьев в лесу: ", grid_search_result.best_params_["n_estimators"])
print ("Лучший критерий: ", grid_search_result.best_params_["criterion"])
print ("Лучшая максимальная глубина дерева: ", grid_search_result.best_params_["max_depth"], "\n")

Лучший результат:  0.8508127648636007
Лучшая количество деревьев в лесу:  10
Лучший критерий:  entropy
Лучшая максимальная глубина дерева:  10 



In [48]:
model = RandomForestClassifier(n_estimators=10, criterion="entropy", max_depth=10, random_state=42)
model.fit(X, y)

predictions = model.predict(X_test)
predictions = predictions.astype(int)

In [49]:
output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
output.to_csv("submission.csv", index=False)